#                                            DIRECTIONAL DRILLING 
##                                           *DIRECTIONAL WELLS PROFILES AND DIRECTIONAL WELLS TRAYECTORIES*

***

![well](Resources/Well_prof.jpg)

# Python Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import namedtuple
from math import radians, isclose, acos, asin, cos, sin, tan, atan, degrees, sqrt

# *Directional Wells Profiles*

## *Slant Well Profile (J Type)*

![j](Resources/j_prof.png)

In [2]:
Data = namedtuple("Input", "TVD KOP BUR DH")
Output = namedtuple("Output", "R Theta TVD_EOB Md_EOB Dh_EOB Tan_len Md_total")

def well_J(data:Data, unit='ingles') -> Output:
    tvd = data.TVD
    kop = data.KOP
    bur = data.BUR
    dh = data.DH
    # Curvature Radio
    if unit == 'ingles':
        R = 5729.58 / bur
    else:
        R = 1718.87 / bur

    # DC Segment   
    if dh > R:
        dc = dh - R
        
    elif dh < R:
        dc =   R - dh

    # DO Segment
    do = tvd - kop

    # DOC Angle
    doc = degrees(atan(dc / do))
    
    # OC Segment
    oc = sqrt(dc**2 + do**2)

    # BOC Angle
    boc = degrees(acos(R / oc))

    # BOD Angle
    if R < dh:
        bod = boc - doc
        
    elif R > dh:
        bod = boc + doc

    # Theta Calculation
    theta = 90 - bod

    # TVD a EOB 
    tvd_eob = kop + abs(R * sin(radians(theta)))

    # MD a EOB
    if unit == 'ingles':
        md_eob = kop + (theta / bur) * 100
        
    else:
        md_eob = kop + (theta / bur) * 30
        
    # DH a EOB
    dh_eob = R - R * cos(radians(theta))

    # BC Segment
    tan_len = sqrt(oc**2 - R**2)

    # MD Total
    if unit == 'ingles':
        md_total = kop + (theta / bur) * 100 + tan_len
    
    else:
        md_total = kop + (theta / bur) * 30 + tan_len
        
    return Output(R=R, Theta=theta, TVD_EOB=tvd_eob, Md_EOB=md_eob, Dh_EOB=dh_eob, \
                  Tan_len=tan_len, Md_total=md_total)

## *Ejercicio 1*

In [3]:
# data
tvd = 8000 #ft
kop = 500 #ft
bur = 2 #o/100ft
dh = 970.8 #ft

In [4]:
trajectory_J = well_J(Data(tvd, kop, bur, dh))

In [5]:
trajectory_J

Output(R=2864.79, Theta=7.564230623470863, TVD_EOB=877.1139517978513, Md_EOB=878.2115311735431, Dh_EOB=24.929649303260703, Tan_len=7185.414140882904, Md_total=8063.625672056447)

In [6]:
names = ['R', 'theta', 'tvd_EOB', 'Md_EOB', 'Dh_EOB', 'Lengh_tan', 'Md_Total']
for param, value in zip(names, trajectory_J):
    if param == 'theta':
        print(f"{param} -> {value:.3f} degrees")
        
    else:
        print(f"{param} -> {value:.3f} ft")

R -> 2864.790 ft
theta -> 7.564 degrees
tvd_EOB -> 877.114 ft
Md_EOB -> 878.212 ft
Dh_EOB -> 24.930 ft
Lengh_tan -> 7185.414 ft
Md_Total -> 8063.626 ft


## *S-Type Well Profile*

![s](Resources/s_prof.png)

In [7]:
# Function for S-Type wells

# Function to calculate parameters from a S-Type well
Data_S = namedtuple("Input", "TVD KOP BUR DOR DH")
Output_S = namedtuple("Output", "R1 R2 Theta TVD_EOB Md_EOB Dh_EOB Tan_len Md_SOD TVD_SOD Dh_SOD Md_total")

def well_S(data:Data_S, unit='ingles'):
    tvd = data.TVD
    kop = data.KOP
    bur = data.BUR
    dor = data.DOR
    dh = data.DH

    # R1 and R2 Calculation
    if unit == 'ingles':
        R1 = 5729.58 / bur
        R2 = 5729.58 / dor
        
    else:
        R1 = 1718.87 / bur
        R2 = 1718.87 / dor

    # FE Segment    
    if dh > (R1 + R2):
        fe = dh - (R1 + R2)
        
    elif dh < (R1 + R2):
        fe = R1 - (dh - R2)

    # EO Segment       
    eo = tvd - kop

    # FOE Angle
    foe = degrees(atan(fe / eo))

    # OF Segment
    of = sqrt(fe**2 + eo**2)

    # FG Segment
    fg = R1 + R2

    # FOG Angle
    fog = degrees(asin(fg / of))

    # Theta Angle
    theta = fog - foe

    # TVD a EOB
    tvd_eob = kop + R1 * sin(radians(theta))

    # MD a EOB
    if unit == 'ingles':
        md_eob = kop + (theta / bur) * 100
        
    else:

        md_eob = kop + (theta / bur) * 30

    # DH a EOB
    dh_eob = R1 - abs(R1 * cos(radians(theta)))

    # OG Segment
    tan_len = sqrt(of**2 - fg**2)

    # MD a SOD
    if unit == 'ingles':
        md_sod = kop + (theta / bur) * 100 + tan_len
        
    else:
         md_sod = kop + (theta / bur) * 30 + tan_len

    # TVD a SOD    
    tvd_sod = tvd_eob + tan_len * abs(cos(radians(theta)))

    # DH a SOD
    dh_sod = dh_eob + abs(tan_len * sin(radians(theta)))

    # MD Total
    if unit == 'ingles':
        md_total = kop + (theta / bur) * 100 + tan_len + (theta / dor) * 100
    
    else:
        md_total = kop + (theta / bur) * 30 + tan_len + (theta / dor) * 30

    
    output_S = Output_S(R1=R1, R2=R2, Theta=theta, TVD_EOB=tvd_eob, Md_EOB=md_eob, Dh_EOB=dh_eob, \
                    Tan_len=tan_len, Md_SOD=md_sod, TVD_SOD=tvd_sod, Dh_SOD=dh_sod, Md_total=md_total)

    names = ['R1', 'R2', 'theta', 'tvd_EOB', 'Md_EOB', 'Dh_EOB', 'Lengh_tan', 'Md_SOD', 'tvd_SOD', 'Dh_SOD', 'Md_Total']
    for param , value in zip(names, output_S):
        if unit == 'ingles':
            if param == 'theta':
                print(f"{param} -> {value:.2f} degrees")
            else:
                print(f"{param} -> {value:.2f} ft")
                
        else:
             if param == 'theta':
                print(f"{param} -> {value:.2f} degrees")
             else:
                print(f"{param} -> {value:.2f} m")

# *Ejercicio 2*

In [8]:
# Data
kop = 6084 #ft
tvd = 12000 #ft
bur = 3 #o/100ft
dor = 2 #o/ft
dh = 3500 #ft

In [9]:
trajectory_S = well_S(Data_S(tvd, kop, bur, dor, dh))

R1 -> 1909.86 ft
R2 -> 2864.79 ft
theta -> 39.93 degrees
tvd_EOB -> 7309.85 ft
Md_EOB -> 7415.01 ft
Dh_EOB -> 445.33 ft
Lengh_tan -> 3718.40 ft
Md_SOD -> 11133.41 ft
tvd_SOD -> 10161.22 ft
Dh_SOD -> 2832.01 ft
Md_Total -> 13129.93 ft


## *Horizontal Well Profiles*

![hor](Resources/Horizontal_prof.jpg)

In [10]:
# Function for horizontal wells

# Function to calculate parameters of a Horizontal Well
Data_H = namedtuple("Input", "TVD KOP BUR1 BUR2 DH")
Output_H = namedtuple("Output", "R1 R2 Theta TVD_EOB1 Md_EOB1 Dh_EOB1 Tan_len Md_SOB2 Md_total")

def well_H(data:Data_H, unit='ingles'):
    # Define the variables
    tvd = data.TVD
    kop = data.KOP
    bur1 = data.BUR1
    bur2 = data.BUR2
    dh = data.DH
    
    # Define curvature radius
    if unit == 'ingles':
        R1 = 5729.58 / bur1
        R2 = 5729.58 / bur2
    else:
        R1 = 1718.87 / bur1
        R2 = 1718.87 / bur2

    # EG Segment
    eg = (tvd - kop) - R2

    # EO Segment
    eo = dh - R1

    # GOE Angle
    goe = degrees(atan(eg / eo))

    # OG Segment
    og = sqrt(eg**2 + eo**2)

    # OF Segment
    of = R1 - R2

    # GOF Angle
    gof = degrees(acos(of / og))
    
    # angle of tangent section
    theta = 180 - goe - gof
    
    # tvd @ EOB1
    tvd_eob1 = kop + (R1 * sin(radians(theta)))
    #MD @ EOB1
    if unit == 'ingles':
        md_eob1 = kop + (theta / bur1) * 100
    else:
        md_eob1 = kop + (theta / bur1) * 30
    # dh @ EOB1
    dh_eob1 = R1 - (R1 * cos(radians(theta)))
    # BC segment 
    tan_len = sqrt(og**2 - of**2)
    # MOD a SOB2
    if unit == 'ingles':
        md_sob2 = kop + (theta / bur1) * 100 + tan_len
    else:
        md_sob2 = kop + (theta / bur1) * 30 + tan_len
    # MD total
    if unit == 'ingles':
        md_total = kop + (theta / bur1) * 100 + tan_len + ((90 - theta) / bur2) * 100
    else:
        md_total = kop + (theta / bur1) * 30 + tan_len + ((90 - theta) / bur2) * 30
        
    output_H = Output_H(R1=R1, R2=R2, Theta=theta, TVD_EOB1=tvd_eob1, Md_EOB1=md_eob1,\
                        Dh_EOB1=dh_eob1, Tan_len=tan_len, Md_SOB2=md_sob2, Md_total=md_total)

    names = ['R1', 'R2', 'Theta', 'TVD_EOB1', 'Md_EOB1', 'Dh_EOB1', 'Tan_len', 'Md_SOB2', 'Md_total']
    for param , value in zip(names, output_H):
        if unit == 'ingles':
            if param == 'theta':
                print(f"{param} -> {value:.2f} degrees")
            else:
                print(f"{param} -> {value:.2f} ft")
        else:
             if param == 'theta':
                print(f"{param} -> {value:.2f} degrees")
             else:
                print(f"{param} -> {value:.2f} m")

## *Ejercicio 3*

In [11]:
# Data
tvd = 3800 #ft
kop = 2000 #ft
bur1 = 5.73 #o/100ft
bur2 = 9.55 #o/100ft
dh = 1800 #ft

In [12]:
well_H(Data_H(tvd, kop, bur1, bur2, dh))

R1 -> 999.93 ft
R2 -> 599.96 ft
Theta -> 49.79 ft
TVD_EOB1 -> 2763.64 ft
Md_EOB1 -> 2868.96 ft
Dh_EOB1 -> 354.40 ft
Tan_len -> 1385.73 ft
Md_SOB2 -> 4254.69 ft
Md_total -> 4675.72 ft


***